In [1]:
import sqlite3
import numpy as np
import pandas as pd

In [2]:
db_con = sqlite3.connect('data/database.sqlite')
cursor = db_con.cursor()
cursor.execute('SELECT name FROM sqlite_master WHERE type="table"')
print(cursor.fetchall())

[('Reviews',)]


In [8]:
pos_neg_reviews = pd.read_sql_query('SELECT * FROM Reviews WHERE Score !=3 LIMIT 10000', db_con)

In [9]:
def assignPositive(x):
    if(x<3):
        return 0
    return 1

In [10]:
colScore = pos_neg_reviews['Score']
posNeg = colScore.map(assignPositive)
pos_neg_reviews['Score'] = posNeg

In [ ]:
pos_